## LangChain Config

In [4]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'


In [21]:

# os.environ["LANGCHAIN_PROJECT"]= os.getenv('LANGCHAIN_PROJECT')
os.environ["LANGCHAIN_PROJECT"]= "MIRA-local-dev"
os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')
os.environ["GOOGLE_API_KEY"] = os.getenv('GOOGLE_API_KEY')
os.environ["LANGCHAIN_API_KEY"] = os.getenv('LANGCHAIN_API_KEY')

In [22]:
os.getenv('LANGCHAIN_PROJECT')

'MIRA-local-dev'

## MongoDB Config

In [6]:
MONGODB_ATLAS_CLUSTER_URI = os.getenv('MONGODB_ATLAS_CLUSTER_URI')
DB_NAME = os.getenv('DB_NAME')
COLLECTION_NAME = os.getenv('COLLECTION_NAME')
ATLAS_VECTOR_SEARCH_INDEX_NAME = os.getenv('ATLAS_VECTOR_SEARCH_INDEX_NAME')

In [7]:
from langchain_openai import OpenAIEmbeddings
from langchain_mongodb.vectorstores import MongoDBAtlasVectorSearch

vector_search = MongoDBAtlasVectorSearch.from_connection_string(
        MONGODB_ATLAS_CLUSTER_URI,
        f"{DB_NAME}.{COLLECTION_NAME}",
        OpenAIEmbeddings(),
        index_name= ATLAS_VECTOR_SEARCH_INDEX_NAME
    )

## HyDE

### Scientific Paper Passage

In [32]:
from langchain.prompts import ChatPromptTemplate

# HyDE document genration
template = """Please write a scientific paper passage to answer the question
Question: {question}
Passage:"""
prompt_hyde = ChatPromptTemplate.from_template(template)

from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

generate_docs_for_retrieval = (
    prompt_hyde | ChatOpenAI(model="gpt-3.5-turbo", temperature=0) | StrOutputParser() 
)

# Run
question="How can I find partner for the life?"
generate_docs_for_retrieval.invoke({"question":question})

"Finding a life partner is a complex and multifaceted process that involves a combination of biological, psychological, and social factors. Research in the field of evolutionary psychology suggests that humans have evolved to seek out partners who possess traits that are indicative of good genetic fitness, such as physical attractiveness, intelligence, and social status. Additionally, individuals may be drawn to partners who exhibit qualities that complement their own personality traits and values.\n\nIn terms of practical strategies for finding a life partner, research has shown that engaging in activities and social settings that align with one's interests and values can increase the likelihood of meeting a compatible partner. This could include joining clubs or organizations, attending social events, or using online dating platforms. Additionally, building strong social networks and maintaining open communication with friends and family members can also provide opportunities for mee

### MongoDB Retrieval

In [12]:
retrieval_chain = generate_docs_for_retrieval | vector_search.as_retriever(search_kwargs={"k":5})
retireved_docs = retrieval_chain.invoke({"question":question})
retireved_docs

[Document(metadata={'_id': {'$oid': '6694b846ff5fa8565a002678'}, 'source': './training/ask me How to find my passion.pdf', 'page': 0, 'start_index': 2363}, page_content="then that's how you start finding your passion. The other side of it is to try to find the passion \nin the job you do. If you're a people person, maybe start finding the passion around \nconnecting with people in the job that you're already doing. If you like to be artistic, maybe in \nthe same job you're doing you can take on a little bit more of being artistic.  \nSo it's in you definitely, you just haven't paid attention, the right attention to what it is that \nyou're naturally  great at, something that you just can gravitate to all the time. You enjoy it, \nyou don't get tired of it. Just observe, it should come to you. Thank you."),
 Document(metadata={'_id': {'$oid': '6694b846ff5fa8565a002676'}, 'source': './training/ask me How to find my passion.pdf', 'page': 0, 'start_index': 795}, page_content="most people d

### Generation

In [8]:
# RAG RAAG
template = """You are an empathetic psychotherapist, parenting counselor, life-coach and mental health expert. 
Be empathetic while answering if the question includes some bad situation. 
Answer the following question based on the context given below, explain each point in a simple language.
Don't look for the answers from other resources. If you don't find the answer from the context, just simply answer: 
"I don't know" or "I didn't understand this, could you please provide some more information on it".

Context: {context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

final_rag_chain = (
    prompt
    | ChatOpenAI(model="gpt-3.5-turbo",temperature=0)
    | StrOutputParser()
)

final_rag_chain.invoke({"context":retrieved_docs,"question":question})

'To find your passion, you can follow these steps:\n\n1. Reflect on your interests: Take some time to think about what activities or hobbies you naturally gravitate towards when you have free time. What do you enjoy doing without feeling tired or bored? This can give you clues about what you are passionate about.\n\n2. Experiment with different activities: Try out new things and see what resonates with you. Explore different hobbies, volunteer opportunities, or creative outlets to see what sparks your interest.\n\n3. Pay attention to feedback: Listen to what others say about your strengths and talents. Sometimes, people around you can see your passion more clearly than you can. If multiple people mention that you excel in a certain area, it could be a sign of your passion.\n\n4. Consider how you can incorporate your passion into your current job: If you are unable to switch careers immediately, think about how you can bring elements of your passion into your current job. Look for ways 

In [10]:
from langchain.prompts import PromptTemplate
from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
)

In [24]:

system_template = """You are an empathetic psychotherapist, parenting counselor, life-coach and mental health expert. 
You need to provide the solution to the user's query.
Following are the instructions how you need to answer the user's query:
 
1. Don't look for the answers from other resources. If you don't find the answer from the context, just simply answer: 
"I don't know" or "I didn't understand this, could you please provide some more information on it".

2. For a positive question, acknowledge the positivity and then give the answer. For example: 
    (a) it is so nice to see that you are in a place in life that you want to _____.
    (b) It is great to see that you are moving to the next phase of your life.
    (c) it is wonderful to know you're ready to excel in different areas of your life.

3. If it's a negative or painful question then empathize with the person that you know and you understand their pain or what they're going through 
and then give the answer. For example: We know that it must be very difficult to deal with ____.
"""

human_template = """
Answer the given question based on the context given below and keeping the instruction in mind, explain each point in a simple language.


Context: {context}

Question: {question}
"""
messages = [
    SystemMessagePromptTemplate.from_template(system_template),
    HumanMessagePromptTemplate.from_template(human_template),
]
CHAT_PROMPT = ChatPromptTemplate.from_messages(messages)

In [34]:
final_rag_chain = (
    CHAT_PROMPT
    | ChatOpenAI(model="gpt-3.5-turbo",temperature=0)
    | StrOutputParser()
)

final_rag_chain.invoke({"context":retrieved_docs,"question":question})

"It's great to see that you are looking to find a partner for life. Here are some simple steps you can follow to find a fulfilling relationship:\n\n1. **Know What You Want**: Take some time to think about the type of relationship you desire. Consider creating a list of qualities you want in a partner and in the relationship.\n\n2. **Put Yourself Out There**: Don't be afraid to socialize and meet new people. Join clubs, classes, or groups where you can interact with others who share your interests.\n\n3. **Practice Communication**: Communication is key in any relationship. Practice talking to people, asking questions, and actively listening to others.\n\n4. **Learn from Others**: Talk to friends or read books about relationships. Learning from others' experiences can give you valuable insights.\n\n5. **Be Open to Rejection**: Rejection is a normal part of the dating process. Don't be discouraged by it. Keep trying and stay positive.\n\n6. **Take Your Time**: Finding the right partner ta